In [1]:
import pandas as pd
df_train = pd.read_csv('train.csv', encoding='utf-8')
df_test = pd.read_csv('test.csv', encoding='utf-8')

In [2]:
import re
def preprocessor(text):
    text = re.sub('[\W]+', ' ', text.lower())
    return text

In [3]:
df_train['text'] = df_train['text'].apply(preprocessor)

In [4]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df_train['author'] = class_le.fit_transform(df_train['author'].values)

In [5]:
from sklearn.model_selection import train_test_split
X, y = df_train.iloc[:, df_train.columns.get_loc('text')].values, \
       df_train.iloc[:, df_train.columns.get_loc('author')].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   stratify=y)

In [6]:
def tokenizer(text):
    return text.split()

In [7]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [8]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
def tokenizer_lemma(text):
    return [lemma.lemmatize(word) for word in text.split()]

In [9]:
import pickle
import os
bag_1gram_lr_pipe = pickle.load(open(os.path.join('pkl_objects', 'bag_1gram_lr_pipe.pkl'), 'rb'))
bag_2gram_lr_pipe = pickle.load(open(os.path.join('pkl_objects', 'bag_2gram_lr_pipe.pkl'), 'rb'))

tfidf_1gram_lr_pipe = pickle.load(open(os.path.join('pkl_objects', 'tfidf_1gram_lr_pipe.pkl'), 'rb'))

In [10]:
from sklearn.ensemble import VotingClassifier

voting_hard_clf = VotingClassifier(
                                estimators=[
                                    ('bag_1gram', bag_1gram_lr_pipe), 
                                    ('bag_2gram', bag_2gram_lr_pipe), 
                                    ('tfidf_1gram', tfidf_1gram_lr_pipe)
                                ],
                                voting='hard')

voting_soft_clf = VotingClassifier(
                                estimators=[
                                    ('bag_1gram', bag_1gram_lr_pipe), 
                                    ('bag_2gram', bag_2gram_lr_pipe), 
                                    ('tfidf_1gram', tfidf_1gram_lr_pipe)
                                ],
                                voting='soft')

voting_hard_clf.fit(X_train, y_train)
voting_soft_clf.fit(X_train, y_train)

all_clf = [bag_1gram_lr_pipe, 
           bag_2gram_lr_pipe, 
           tfidf_1gram_lr_pipe, 
           voting_hard_clf, 
           voting_soft_clf]

In [11]:
from sklearn.metrics import accuracy_score
for clf in (all_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf, accuracy_score(y_test, y_pred))

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['i', 'me'...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]) 0.700374531835206
Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words=None,
        stri...alty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]) 0.48927477017364657
Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_e

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


VotingClassifier(estimators=[('bag_1gram', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1,...y='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None) 0.716207013959823
VotingClassifier(estimators=[('bag_1gram', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1,...y='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
